In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
import json
import requests
from datetime import datetime

In [2]:
scope="playlist-modify-public playlist-modify-private"

In [3]:
credentials = "spotify_keys.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)

In [4]:
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect_uri"]
weather_api_key = api_tokens["weather_api_key"]
playlist_id = api_tokens["playlist_id"]
username = api_tokens["username"]

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri=redirect_uri,
    scope="playlist-modify-public playlist-modify-private"
    ))

In [5]:
city = "London"  

weather_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}&units=metric"
weather_response = requests.get(weather_url)
weather_data = weather_response.json()

weather_condition = weather_data['weather'][0]['main'].lower()
temperature = weather_data['main']['temp']

In [6]:
weather_music = {
    'thunderstorm': ['rock', 'metal'],
    'rain': ['jazz', 'lofi'],
    'snow': ['classical', 'ambient'],
    'clear': ['pop', 'dance'],
    'clouds': ['indie', 'alternative'],
    'mist': ['ambient', 'downtempo'],
    'drizzle': ['jazz', 'lofi'],
    'fog': ['ambient', 'classical']
}

In [7]:
if weather_condition in weather_music:
    genres = weather_music[weather_condition]
else:
    genres = ['pop']  

In [8]:
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)
sp = spotipy.Spotify(auth=token)

tracks = []
for genre in genres:
    results = sp.recommendations(
        seed_genres=[genre],
        limit=10
    )
    tracks.extend([track['uri'] for track in results['tracks']])

In [9]:
print(tracks)

['spotify:track:6XwfbweA3ppZC6sI6GzC1G', 'spotify:track:65KWSO41BTzyBEthvaMqGS', 'spotify:track:7KsZHCfOitA5V9oQYVdltG', 'spotify:track:5EA06TsQmqmQXZiEhr1AaM', 'spotify:track:1p2TWJxUlHE5JR0sE6E1d7', 'spotify:track:71RPveF6c3dgG4tiljQw9K', 'spotify:track:48HT7Zzd3We4ZjG82lsbdK', 'spotify:track:6M3u70GWuRDLcyvpPJEgzk', 'spotify:track:5Q6Xzp25aXYxNJ4B6vBkcf', 'spotify:track:4yYvWFUWMaKMXtoJKJnmZk', 'spotify:track:1YI0uK36eupTmw9F8kHysr', 'spotify:track:3HPfwpfUtcdyJPvlLtXtNg', 'spotify:track:36tghkPg1AO4HsAzo6YpPp', 'spotify:track:3YuaBvuZqcwN3CEAyyoaei', 'spotify:track:77eRSoQbDt4qHlAhUqovmy', 'spotify:track:0W7AbEauB7cP4pidLclApe', 'spotify:track:1z7ipCbtUKz3Nepd2r39bc', 'spotify:track:1RwwmiVtLAtPmxAqKVfwgG', 'spotify:track:12qZHAeOyTf93YAWvGDTat', 'spotify:track:7C6NcqS3fNDXLgP5L2Kbgp']


In [10]:
tracks = list(set(tracks))[:20]

sp.playlist_replace_items(playlist_id, tracks)

playlist_info = f"Weather-based playlist for {city}. Weather: {weather_condition}. Updated: {datetime.now().strftime('%Y-%m-%d %H:%M')}"
sp.playlist_change_details(playlist_id, description=playlist_info)

In [11]:
print(f"Weather Condition: {weather_condition}")
print(f"Selected Genres: {genres}")
print(f"Number of tracks added: {len(tracks)}")

Weather Condition: clouds
Selected Genres: ['indie', 'alternative']
Number of tracks added: 20


In [12]:
playlist_tracks = sp.playlist_tracks(playlist_id)
print("\nAdded Tracks:")
for idx, item in enumerate(playlist_tracks['items'], 1):
    track = item['track']
    print(f"{idx}. {track['name']} - {track['artists'][0]['name']}")


Added Tracks:
1. Everything Now - Arcade Fire
2. I Wanna Get Better - Bleachers
3. House of Gold - Twenty One Pilots
4. Like a Stone - Audioslave
5. Sirens - Pearl Jam
6. All The Small Things - blink-182
7. Arms - The Paper Kites
8. Best to You - Blood Orange
9. Saw You In a Dream - The Japanese House
10. Nice to Know You - Incubus
11. Hunger Strike - Temple Of The Dog
12. Shine - Benjamin Francis Leftwich
13. Magician (Suture) - Milo
14. Shuffle - Bombay Bicycle Club
15. Coming Home - Leon Bridges
16. Sunday Morning - Maroon 5
17. Luna - 2011 Remaster - The Smashing Pumpkins
18. Cherry - Chromatics
19. Baby (Bonus Track) - Ariel Pink
20. Just Like Heaven - The Cure


In [14]:
import webbrowser
# Opens a specific Spotify playlist
webbrowser.open(f'https://open.spotify.com/playlist/{playlist_id}')


True